In [1]:
import serial
import csv
import os
from datetime import datetime, timedelta
import time
from pathlib import Path

In [2]:

def reset_esp32(port, baudrate=921600):
    """Resets an ESP32 using the RTS and DTR lines of the serial port.

    Args:
        port: The serial port (e.g., "COM3" on Windows, "/dev/ttyUSB0" on Linux).
        baudrate: The baud rate (typically 115200 for communication,
                  but can be higher for flashing).
    """
    try:
        with serial.Serial(port, baudrate) as ser:
            # Method 1 (esptool.py compatible):
            ser.dtr = False  # DTR = 0: Set EN (reset) to HIGH
            ser.rts = True   # RTS = 1, Set IO0 (GPIO0) to LOW: BOOT mode
            time.sleep(0.1)
            ser.dtr = True   # DTR = 1, Set EN to LOW: Reset chip
            time.sleep(0.1)  # Wait for the chip to reset
            ser.dtr = False  # DTR = 0, set EN to HIGH
            ser.rts = False # Set IO0 to HIGH, back to normal mode.
            time.sleep(0.1)

            # Method 2 (Simple DTR toggle - might work in some cases):
            # ser.dtr = True   # Assert DTR (often connected to reset)
            # time.sleep(0.1)
            # ser.dtr = False  # De-assert DTR
            # time.sleep(0.1)
            print(f"ESP32 reset via DTR/RTS on port {port}")

    except serial.SerialException as e:
        print(f"Error resetting ESP32: {e}")


In [12]:
def collect_csi_data(filepath, port, baudrate = 921600, seconds = 100, minutes = 0, hours = 0, temp_data = False):
    with serial.Serial(port, baudrate) as ser:
        header = "type,seq,mac,rssi,rate,sig_mode,mcs,bandwidth,smoothing,not_sounding,aggregation,stbc,fec_coding,sgi,noise_floor,ampdu_cnt,channel,secondary_channel,local_timestamp,ant,sig_len,rx_state,len,first_word,data,datetime"
        
        start_time = datetime.now()
        
        filename = filepath
        if temp_data:
            temp_file_path = temp_data
        
        if not os.path.exists(filename):
            with open(filename, 'w', encoding = 'utf-8') as f:
                f.write(header + '\n')
                
        
        
        num_collected = 0

        end_time = timedelta(seconds = seconds, minutes = minutes, hours = hours)
        
        if ser.in_waiting > 0:
            ser.reset_input_buffer()
        
        while (datetime.now() - start_time) <= end_time:
            if ser.in_waiting > 0:
                try:
                    line = ser.readline().decode('utf-8').strip()
                except:
                    print("DECODE FAILED")
                    continue
                line = line + f',{datetime.now()}'
                if not line.startswith("CSI_DATA"):
                    continue
                with open(filename, 'a', newline = '', encoding = 'utf-8') as f:
                    f.write(line + '\n')       
        
                num_collected += 1

                if temp_data:
                    if num_collected % 5 == 0:
                        with open(temp_file_path, 'w', newline = '', encoding = 'utf-8') as f:
                            f.write(header + '\n')
                            f.write(line + '\n')  
                
                if num_collected % 100 == 0:
                    print(f"{datetime.now()} Number of CSI data collected : {num_collected}")
                    print(f"ser_inwaiting : {ser.in_waiting}")
        if temp_data:
            os.remove(temp_file_path)
        
        print(f"Total : {num_collected}")
    

In [13]:
import serial
import os
import time # Using time.time() for potentially faster checks
from datetime import datetime, timedelta
import sys # For flushing stdout
import io # For potential memory buffering if needed, though list is usually fine

# --- Performance Tuning Configuration ---

# Increased buffer size: Write to disk less frequently. Adjust based on RAM.
# 100 packets/sec * 10 seconds = 1000 packets. Let's buffer more.
# Consider average line length * BUFFER_SIZE for memory usage.
BUFFER_SIZE = 5000

# How often (in loop iterations) to check the current time against the end time.
# Checking less often reduces overhead but might overshoot the duration slightly.
TIME_CHECK_INTERVAL = 200 # Check time every 200 received packets

# How often (in collected packets) to update the temporary file (if enabled)
TEMP_FILE_UPDATE_INTERVAL = 50 # Update temp file every 50 packets (~0.5 sec)

# How often (in collected packets) to print status updates
STATUS_UPDATE_INTERVAL = 5000 # Print status every 5000 packets (~50 sec)

# Serial read timeout (seconds). Keep it relatively small to avoid blocking
# but large enough to likely get a line if data is flowing.
SERIAL_TIMEOUT = 0.05 # 50 milliseconds

# --- Constants ---
CSI_DATA_PREFIX = b"CSI_DATA" # Use bytes for faster startswith check

def collect_csi_data_optimized(filepath, port, baudrate=921600, seconds=100, minutes=0, hours=0, temp_data_path=None):
    """
    Collects CSI data from a serial port at high rates, optimized for performance.

    Args:
        filepath (str): Path to the main file where CSI data will be saved.
        port (str): The serial port name (e.g., 'COM3' or '/dev/ttyUSB0').
        baudrate (int): The serial baud rate. Defaults to 921600.
        seconds (int): Duration to collect data for (in seconds).
        minutes (int): Duration to collect data for (in minutes).
        hours (int): Duration to collect data for (in hours).
        temp_data_path (str, optional): Path to a temporary file to store the
                                         most recent CSI packet periodically.
                                         If None, this feature is disabled.
                                         Defaults to None.
    """
    header = "type,seq,mac,rssi,rate,sig_mode,mcs,bandwidth,smoothing,not_sounding,aggregation,stbc,fec_coding,sgi,noise_floor,ampdu_cnt,channel,secondary_channel,local_timestamp,ant,sig_len,rx_state,len,first_word,data,datetime\n"
    total_duration = timedelta(seconds=seconds, minutes=minutes, hours=hours)
    start_time = datetime.now()
    # Using time.monotonic() might be slightly more efficient for interval timing if needed,
    # but datetime is required for the end condition check.
    end_time = start_time + total_duration

    print(f"Starting HIGH RATE CSI data collection for {total_duration}...")
    print(f"Targeting ~100 packets/sec.")
    print(f"Saving data to: {filepath}")
    if temp_data_path:
        print(f"Updating latest packet temp file ({temp_data_path}) every {TEMP_FILE_UPDATE_INTERVAL} packets.")
        temp_dir = os.path.dirname(temp_data_path)
        if temp_dir and not os.path.exists(temp_dir):
             os.makedirs(temp_dir)

    num_collected = 0
    loop_counter = 0
    line_buffer = []
    last_valid_line_for_temp = "" # Store the last line for less frequent temp writes

    try:
        # Ensure output directory exists
        output_dir = os.path.dirname(filepath)
        if output_dir and not os.path.exists(output_dir):
            os.makedirs(output_dir)

        file_existed = os.path.exists(filepath)

        # Open resources outside the loop
        # Consider 'buffering=-1' for OS default buffering (usually good)
        with serial.Serial(port, baudrate, timeout=SERIAL_TIMEOUT) as ser, \
             open(filepath, 'a', encoding='utf-8', newline='', buffering=-1) as data_file, \
             (open(temp_data_path, 'w', encoding='utf-8', newline='') if temp_data_path else open(os.devnull, 'w')) as temp_f:

            # --- File Initialization ---
            if not file_existed or os.path.getsize(filepath) == 0:
                data_file.write(header)
                data_file.flush() # Ensure header is written

            if temp_data_path:
                temp_f.write(header) # Write header to temp file once
                temp_f.flush()

            # --- Clear Initial Serial Buffer ---
            if ser.in_waiting > 0:
                print(f"Clearing {ser.in_waiting} bytes from initial serial buffer...")
                # Read existing bytes quickly, readline might be too slow here if buffer is huge
                ser.read(ser.in_waiting)
                print("Buffer cleared.")

            # Get current time once before loop for first check
            current_time = datetime.now()

            # --- Main Collection Loop ---
            while current_time < end_time:
                # Read bytes directly
                line_bytes = ser.readline()

                # Check if data was actually received (timeout returns b'')
                if not line_bytes:
                    # No data received (timeout), just check time condition if needed
                    loop_counter += 1
                    if loop_counter % TIME_CHECK_INTERVAL == 0:
                         current_time = datetime.now()
                    continue # Go to next iteration quickly

                # --- Process Received Data ---
                # Check prefix using bytes (faster)
                if line_bytes.startswith(CSI_DATA_PREFIX):
                    try:
                        # Decode only if it's potential CSI data
                        line = line_bytes.decode('utf-8').strip()
                        # Get timestamp *only* for valid lines
                        timestamp = datetime.now() # System call, minimize usage
                        current_time = timestamp # Update current time since we just called it

                        formatted_line = f"{line},{timestamp}\n"
                        line_buffer.append(formatted_line)
                        num_collected += 1
                        last_valid_line_for_temp = formatted_line # Keep track for temp file

                        # --- Batch Write to Main File ---
                        if len(line_buffer) >= BUFFER_SIZE:
                            data_file.writelines(line_buffer)
                            # Avoid flushing here unless absolutely necessary - rely on OS buffering
                            # data_file.flush()
                            line_buffer.clear()

                        # --- Periodic Temp File Update ---
                        if temp_data_path and (num_collected % TEMP_FILE_UPDATE_INTERVAL == 0):
                            temp_f.seek(0)
                            temp_f.write(header)
                            if last_valid_line_for_temp: # Ensure we have a line
                                temp_f.write(last_valid_line_for_temp)
                            temp_f.truncate()
                            temp_f.flush() # Flush temp file so it's readable externally

                        # --- Periodic Status Update ---
                        if num_collected % STATUS_UPDATE_INTERVAL == 0:
                            elapsed_time = timestamp - start_time
                            rate = num_collected / elapsed_time.total_seconds() if elapsed_time.total_seconds() > 0 else 0
                            print(f"{timestamp} | Coll: {num_collected} | Rate: {rate:.2f} pkt/s | Buf: {len(line_buffer)}/{BUFFER_SIZE} | SerInWait: {ser.in_waiting}")
                            sys.stdout.flush()

                    except UnicodeDecodeError:
                        # Log decode errors less intrusively for high rates
                        if num_collected % 100 == 0: # Log only occasionally
                           print(f"W:{datetime.now()}: UnicodeDecodeError - skipping line.", file=sys.stderr)
                        continue # Skip malformed line

                # --- Time Check (Less Frequent) ---
                # Only check time explicitly every N loops if data wasn't received OR
                # if we haven't updated current_time via timestamping recently.
                # The `current_time = timestamp` line often handles this implicitly.
                else:
                    # Line received but wasn't CSI_DATA
                    loop_counter += 1
                    if loop_counter % TIME_CHECK_INTERVAL == 0:
                         current_time = datetime.now()

            # --- End of Collection ---
            print(f"Collection time ended at {datetime.now()}. Finishing writes...")
            # Write any remaining lines in the buffer
            if line_buffer:
                print(f"Writing final {len(line_buffer)} lines to file...")
                data_file.writelines(line_buffer)
                data_file.flush() # Final flush for main file
                line_buffer.clear()

    except serial.SerialException as e:
        print(f"ERROR: Serial port communication failed: {e}", file=sys.stderr)
    except IOError as e:
        print(f"ERROR: File I/O error: {e}", file=sys.stderr)
    except Exception as e:
        print(f"ERROR: An unexpected error occurred: {e}", file=sys.stderr)
        import traceback
        traceback.print_exc() # Print full traceback for unexpected errors
    finally:
        # --- Cleanup ---
        if temp_data_path and os.path.exists(temp_data_path):
            try:
                os.remove(temp_data_path)
                print(f"Removed temporary file: {temp_data_path}")
            except OSError as e:
                print(f"WARN: Could not remove temporary file {temp_data_path}. Error: {e}", file=sys.stderr)

        end_run_time = datetime.now()
        total_run_duration = end_run_time - start_time
        print(f"--------------------------------------------------")
        print(f"Collection finished at {end_run_time}.")
        print(f"Total actual duration: {total_run_duration}")
        print(f"Total CSI data packets collected: {num_collected}")
        if total_run_duration.total_seconds() > 0:
             final_rate = num_collected / total_run_duration.total_seconds()
             print(f"Average collection rate: {final_rate:.2f} packets/sec")
        print(f"Data saved to: {filepath}")
        print(f"--------------------------------------------------")


In [22]:
PORT = "COM8"
BAUDRATE = "921600"
FILEPATH = r"D:\Matthew\SelfStudy\csi-project\csi-project\data_collection\03_04_2025_LY8_CSI_DATA\stand_walk_valid.csv"


reset_esp32(PORT, BAUDRATE)
for i in range(15):
    print(f"start in {15-i}")
    time.sleep(1)
collect_csi_data(FILEPATH, PORT, BAUDRATE, seconds = 0, minutes = 2)

ESP32 reset via DTR/RTS on port COM8
start in 15
start in 14
start in 13
start in 12
start in 11
start in 10
start in 9
start in 8
start in 7
start in 6
start in 5
start in 4
start in 3
start in 2
start in 1
2025-04-03 21:33:04.503627 Number of CSI data collected : 100
ser_inwaiting : 0
2025-04-03 21:33:08.819004 Number of CSI data collected : 200
ser_inwaiting : 0
2025-04-03 21:33:12.626609 Number of CSI data collected : 300
ser_inwaiting : 0
2025-04-03 21:33:16.514021 Number of CSI data collected : 400
ser_inwaiting : 0
2025-04-03 21:33:20.927345 Number of CSI data collected : 500
ser_inwaiting : 0
2025-04-03 21:33:25.378893 Number of CSI data collected : 600
ser_inwaiting : 0
2025-04-03 21:33:30.309720 Number of CSI data collected : 700
ser_inwaiting : 0
2025-04-03 21:33:34.151407 Number of CSI data collected : 800
ser_inwaiting : 0
2025-04-03 21:33:38.775228 Number of CSI data collected : 900
ser_inwaiting : 0
2025-04-03 21:33:42.348634 Number of CSI data collected : 1000
ser_inwai